## Import Dependencies

In [14]:
import csv
from pathlib import Path
import pandas as pd
import glob
from datetime import datetime, timedelta
import numpy as np
# from sklearn import preprocessing
import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LinearRegression, RidgeCV
# from sklearn.metrics import mean_squared_error
import seaborn as sns
import pandas_profiling as pp
from settings import *
# from settings import DATA_DIR

In [15]:
outputFileName = 'FMPerfProcessed.csv'
DOWNLOAD_DIR = 'D:\Performance_All'

# Get the Single-Family Loan Performance Data Files from Fannie Mae

From the Fannie Mae [website:](https://loanperformancedata.fanniemae.com/lppub/index.html#Single-Family_Loan_Performance_Data_Files) we downloaded the Performance dataset (26GB zip file)

The [Performance file layout](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_File_layout.pdf) was as follows:
![Performance File Layout](images/PerformanceFileLayout.jpg)

In [16]:
# Create mapping for the header of the files based on above file format
allheaderline = ['id','rptPeriod','sellerName','currIntRate','currUPB','loanAge','monMatur','adjMonMatur',\
              'maturDate','mSA','deliqStatus','modFlag','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']
headerline = ['id','rptPeriod','currIntRate','currUPB','loanAge','monMatur',\
              'mSA','deliqStatus','zeroBalCode','zeroBalDate','lastPdInstDate',\
              'forecloDate','dispDate','forecloCost','propRepCost','recovCosts','miscCost','holdTaxCost',\
              'saleProceed','credEnhProceed','repurchProceed','otherForecloProceed','nonIntUPB',\
              'prinForgivBal','repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator']

In [17]:
# Loop through each file from the unzipped download file - which gave a set of 76 individual files

#Create an empty array to hold dataframes to later concatonate
li = []

#Loop through every file in the directory and append into array
for in_path in Path(DOWNLOAD_DIR).glob('Performance_2000Q1.txt'):
    df = pd.read_csv(in_path, sep="|", index_col=None, header=None, usecols=headerline)
    # Perform groupby id and then min(),max() and count() to get one summary line for each unique id
    def loanagg(x):
        d = {}
        d['rptPeriodMin'] = x['rptPeriod'].min()
        d['rptPeriodMax'] = x['rptPeriod'].max()
        d['currIntRateMin'] = x['currIntRate'].min()
        d['currUPBMin'] = x['currUPB'].min()
        d['loanAgeMin'] = x['loanAge'].min()
        d['loanAgeMax'] = x['loanAge'].max()
        d['monMaturMax'] = x['monMatur'].max()
        d['mSA'] = x['mSA'].max()
        d['deliqStatusGoodCnt'] = x['deliqStatus'].mask(df['deliqStatus'].ne(0)).count()
        d['deliqStatusBadCnt'] = x['deliqStatus'].mask(df['deliqStatus'].le(0)).count()
        d['deliqStatusMaxInRow'] = x['deliqStatus'].max()
        d['zeroBalCode'] = x['zeroBalCode'].max()
        d['zeroBalDate'] = x['zeroBalDate'].max()
        d['lastPdInstDate'] = x['lastPdInstDate'].max()
        d['forecloDate'] = x['forecloDate'].max()
        d['dispDate'] = x['dispDate'].max()
        d['forecloCost'] = x['forecloCost'].max()
        d['propRepCost'] = x['propRepCost'].max()
        d['recovCosts'] = x['recovCosts'].max()
        d['miscCost'] = x['miscCost'].max()
        d['holdTaxCost'] = x['holdTaxCost'].max()
        d['saleProceed'] = x['saleProceed'].max()
        d['credEnhProceed'] = x['credEnhProceed'].max()
        d['repurchProceed'] = x['repurchProceed'].max()
        d['otherForecloProceed'] = x['otherForecloProceed'].max()
        d['nonIntUPB'] = x['nonIntUPB'].max()
        d['prinForgivBal'] = x['prinForgivBal'].max()
        d['repurchMakeWholeProceedFlg'] = x['repurchMakeWholeProceedFlg'].max()
        d['forecloPrinWriteOffAmnt'] = x['forecloPrinWriteOffAmnt'].max()
        d['servActivIndicator'] = x['servActivIndicator'].max()
        return pd.Series(d, index=['rptPeriodMin','rptPeriodMax','currIntRateMin','currUPBMin',\
                                  'loanAgeMin','loanAgeMax','monMaturMax','mSA','deliqStatusGoodCnt',\
                                  'deliqStatusBadCnt','deliqStatusMaxInRow','zeroBalCode','zeroBalDate',\
                                  'lastPdInstDate','forecloDate','dispDate','forecloCost','propRepCost',\
                                  'recovCosts','miscCost','holdTaxCost','saleProceed','credEnhProceed',\
                                  'repurchProceed','otherForecloProceed','nonIntUPB','prinForgivBal',\
                                  'repurchMakeWholeProceedFlg','forecloPrinWriteOffAmnt','servActivIndicator'])
    
    df.groupby('id').apply(loanagg)
    li.append(df)

#Concatonate all the array elements into one giant dataframe
df = pd.concat(li, axis=0, ignore_index=True)
df.columns = headerline

In [18]:
df.shape

(9149461, 13)

In [19]:
#Review the columns for datatypes and how many non-null values 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9149461 entries, 0 to 9149460
Data columns (total 13 columns):
id             int64
rptPeriod      object
currIntRate    float64
currUPB        float64
loanAge        int64
monMatur       float64
adjMonMatur    float64
maturDate      object
mSA            int64
deliqStatus    object
modFlag        object
zeroBalCode    float64
forecloDate    object
dtypes: float64(5), int64(3), object(5)
memory usage: 907.5+ MB


In [20]:
df.head()

,id,rptPeriod,currIntRate,currUPB,loanAge,monMatur,adjMonMatur,maturDate,mSA,deliqStatus,modFlag,zeroBalCode,forecloDate
0,100007365142,01/01/2000,8.0,NaN,0,360.0,359.0,01/2030,0,0,N,NaN,NaN
1,100007365142,02/01/2000,8.0,NaN,1,359.0,358.0,01/2030,0,0,N,NaN,NaN
2,100007365142,03/01/2000,8.0,NaN,2,358.0,357.0,01/2030,0,0,N,NaN,NaN
3,100007365142,04/01/2000,8.0,NaN,3,357.0,356.0,01/2030,0,0,N,NaN,NaN
4,100007365142,05/01/2000,8.0,NaN,4,356.0,355.0,01/2030,0,0,N,NaN,NaN


### Convert the origDate,firstPmtDate columns from strings to datetime datatypes
Since this will be such a large dataset I tested a number of methods to change datatypes

In [21]:
df['rptPeriod'] = pd.to_datetime(df['rptPeriod'], format='%m/%d/%Y')
df['maturDate'] = pd.to_datetime(df['maturDate'], format='%m/%Y')

### Filter out records with values we aren't interested in examining:

Using the [Glossary](https://loanperformancedata.fanniemae.com/lppub-docs/FNMA_SF_Loan_Performance_Glossary.pdf) of column names we made the following selections:
![Property Type](images/propertyType.jpg)
* Property type: only want 'SF' - 'Single Family'

In [10]:
propTypeFilter = ['SF']
df = df[df.propType.isin(propTypeFilter)]
df.shape

(30093769, 25)

![Number of Units](images/numUnits.jpg)
* Number of Units: only want '1'

In [11]:
numUnitsFilter = [1]
df = df[df.numUnits.isin(numUnitsFilter)]
df.shape

(29072952, 25)

![Number of Units](images/occType.jpg)
* Number of Units: only want 'P'

In [12]:
occTypeFilter = ['P']
df = df[df.occType.isin(occTypeFilter)]
df.shape

(26581922, 25)

![Number of Units](images/loanPurp.jpg)
* Number of Units: only want 'C,R,U'

In [13]:
loanPurpFilter = ['C','R','U']
df = df[df.loanPurp.isin(loanPurpFilter)]
df.shape

(18724184, 25)

![Number of Units](images/origLoanTerm.jpg)
* Number of Units: only want '360'

In [14]:
origLoanTermFilter = [360]
df = df[df.origLoanTerm.isin(origLoanTermFilter)]
df.shape

(10715674, 25)

### Fill null values with 0 for the following columns
We can't have NaN or Null values for the machine learning part to come

In [15]:
df['mortInsType'] = df['mortInsType'].fillna(0)
df['pMIperct'] = df['pMIperct'].fillna(0)
df['origCLTV'] = df['origCLTV'].fillna(0)

## Calculate the 'Best', 'Worst' and 'Avg' Credit Score
Given that there is a Borrower Credit Score and a Co-Borrower Credit Score and that there are Null values for each we want to create calculated columns where no Null values exist and then drop the original columns

In [16]:
df['bestCreditScore'] = df[['borrCreditScore','coborrCreditScore']].max(axis=1)
df['worstCreditScore'] = df[['borrCreditScore','coborrCreditScore']].min(axis=1)
df['avgCreditScore'] = df[['borrCreditScore','coborrCreditScore']].mean(axis=1)

### Get rid of records where there are NaN or NULL values for:
* 'Borrower Credit Score'
* 'Debt to Income Ratio'
* 'Number of Borrowers'

In [17]:
df = df[df['borrCreditScore'].notna() &\
        df['origDebtIncRatio'].notna() &\
        df['numBorrowers'].notna()]

### Drop all columns where there is only a single value (and coborrower Credit Score)

In [18]:
df = df.drop(['prodType','firstTHBI','relocMortInd','occType','numUnits','origLoanTerm','coborrCreditScore','propType'], axis=1)

### Map text values to integers (Machine Learning will require this later)

![OrigChannel](images/origChannel.jpg)

In [19]:
df['origChannel'] = df['origChannel'].replace(['R','B','C'],[1,2,3])
df['origChannel'] = pd.to_numeric(df['origChannel'])

![Loan Purpose](images/loanPurp.jpg)

In [20]:
df['loanPurp'] = df['loanPurp'].replace(['C','R','U'],[1,2,3])
df['loanPurp'] = pd.to_numeric(df['loanPurp'])

In [21]:
df['loanPurp'].unique()

array([2, 1, 3], dtype=int64)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10464607 entries, 0 to 40887331
Data columns (total 20 columns):
loanIdentifier      int64
origChannel         int64
sellerName          object
origIntRate         float64
origUPB             int64
origDate            datetime64[ns]
firstPmtDate        datetime64[ns]
origLTV             float64
origCLTV            float64
numBorrowers        float64
origDebtIncRatio    float64
borrCreditScore     float64
loanPurp            int64
propState           object
zipCode             int64
pMIperct            float64
mortInsType         float64
bestCreditScore     float64
worstCreditScore    float64
avgCreditScore      float64
dtypes: datetime64[ns](2), float64(11), int64(5), object(2)
memory usage: 1.6+ GB


In [23]:
df.head()

,loanIdentifier,origChannel,sellerName,origIntRate,origUPB,origDate,firstPmtDate,origLTV,origCLTV,numBorrowers,origDebtIncRatio,borrCreditScore,loanPurp,propState,zipCode,pMIperct,mortInsType,bestCreditScore,worstCreditScore,avgCreditScore
0,100007365142,1,"JPMORGAN CHASE BANK, NA",8.000,75000,1999-12-01,2000-02-01,79.0,0.0,1.0,62.0,763.0,2,PA,173,0.0,0.0,763.0,763.0,763.0
25,100094030068,3,"CITIMORTGAGE, INC.",7.750,152000,1999-11-01,2000-01-01,85.0,0.0,1.0,41.0,604.0,2,NV,891,25.0,1.0,604.0,604.0,604.0
30,100118647578,2,"JPMORGAN CHASE BANK, NA",7.750,106000,1999-11-01,2000-01-01,80.0,0.0,2.0,27.0,652.0,2,OR,972,0.0,0.0,696.0,652.0,674.0
32,100122555751,1,"GMAC MORTGAGE, LLC",8.250,191000,2000-02-01,2000-04-01,78.0,0.0,1.0,52.0,676.0,1,CA,921,0.0,0.0,676.0,676.0,676.0
34,100123116925,3,"BANK OF AMERICA, N.A.",7.875,240000,1999-12-01,2000-02-01,74.0,0.0,3.0,40.0,718.0,2,CA,940,0.0,0.0,718.0,685.0,701.5


In [24]:
# Check if updated files from Fannie Mae have created duplicate entries for a given loan Identifier 
#(number should equal the number of records if there are no duplicates)
df.groupby('loanIdentifier').loanIdentifier.nunique().sum()

10464607

In [25]:
sellerNames = df['sellerName'].unique()
statesList = df['propState'].unique()
#sellerNames

In [26]:
#sellerNames

In [33]:
pathparts = (DATA_DIR,outputFileName)
outpath = "/".join(pathparts)
outpath

'data/FMAcqProcessed.csv'

In [ ]:
df.to_csv(outpath)